## PROJE SORUMLULARI

> **AD SOYAD : DOĞUKAN SÜRÜCÜ , ÖĞRENCİ NUMARASI: 200403617**

> **AD SOYAD : FIRAT GÜMÜŞ , ÖĞRENCİ NUMARASI: 200403647**

In [1]:
import pandas as pd
import numpy as np

## DECİSİON TREES NEDİR ?

> Decision Tree genellikle sınıflandırma problemleri için kullanılan bir gözetimli öğrenme algoritmasıdır. Regresyon içinde kullanılabilir. Yüksek doğruluk, kararlılık ve yorumlanma kolaylığına sahiptir. Doğrusal olmayan ilişkileride oldukça iyi tespit edebilir.

> Aşamalı bir işlevdir, çünkü her bölmede algoritma, bölünmenin kalitesine veya ne kadar saf olduğuna (daha az hata) bağlı olarak o noktada hangi özelliğin kullanılacağını hesaplar.

## AŞAMA AŞAMA DECİSİON TREES

* Veriseti içerisinden bir özellik seçilir.
* Bu özelliğin, verinin ayrılmasındaki önemi hesaplanır.
* Yukarıdaki işlem, bölme işini en iyi yapan özelliği bulana kadar tekrar edilir.
* En iyi bölen özellik bulununca bir kural oluşturulur.
* Yukarıda ki adımlar önem sırasına göre tüm özellikler için tekrarlanır.
* Oluşturulan tüm kurallar verinin ayrılmasındaki önem sırasına göre sıralanır.
* En önemli ayrıştırıcı özellik kök olur, önemi az olana doğru giderken yapraklara tek tek ulaşılır.

**ENTROPY:** <span style='color:orange'>**Rastgeleliğe, belirsizliği ve beklenmeyen durumun ortaya çıkma olasılığını gösterir. Eğer örnekler tamamı düzenli / homojen ise entropisi sıfır olur. Eğer değerler birbirine eşit ise entropi 1 olur. Örneğin Futbol Oyna hepsi “Evet” veya “Hayır” olsa entropi sıfır olurdu. Entropi formülasyonu:**</span>


$$Entropy(x) = - \sum ( P(x=k)) * \operatorname{Log}_2 P(x=k))$$

**INFORMATİON GAİN (Bilgi Kazanımı):**
<span style='color:orange'>**Bilgi kazanımı, bir veri setini bir özellik üzerinde böldükten sonra tüm entropiden çıkarmaya dayanır. Entropinin küçük değer içermesi durumunda özelliğin önemi Decision Tree algoritması ID3 için artmaktadır. Diğer taraftan 1’e yaklaştıkça özelliğinin önemi azalır. Ancak information gain’de olay tam tersidir ve bu açıdan entropinin tersi gibi düşünülebilir.  Decision Tree inşa edilirken en yüksek değerleri information gain’e sahip özellik seçilir.**</span>

$$Information Gain(feature) = Entropy(dataset) - Entropy(feature)$$

In [2]:
df = pd.DataFrame(
    {
       "Outlook": ['Rainy', 'Rainy', 'Overcast', 'Sunny', 'Sunny', 'Sunny',
       'Overcast', 'Rainy', 'Rainy', 'Sunny', 'Rainy', 'Overcast',
       'Overcast', 'Sunny'],
        
       "Temp": ['Hot', 'Hot', 'Hot', 'Mild', 'Cool', 'Cool', 'Cool', 'Mild',
       'Cool', 'Mild', 'Mild', 'Mild', 'Hot', 'Mild'],
        
       "Humidity": ['High', 'High', 'High', 'High', 'Normal', 'Normal', 'Normal',
       'High', 'Normal', 'Normal', 'Normal', 'High', 'Normal', 'High'],
        
       "Windy": ['False', 'True', 'False', 'False', 'False', 'True', 'True', 'False', 'False',
       'False', 'True', 'True', 'False', 'True'],
        
       "Play Golf": ['No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes',
       'Yes', 'Yes', 'Yes', 'No']
    },
    index=[i for i in range(14)],
)

In [3]:
eps = np.finfo(float).eps

In [4]:
df

,Outlook,Temp,Humidity,Windy,Play Golf
0,Rainy,Hot,High,False,No
1,Rainy,Hot,High,True,No
2,Overcast,Hot,High,False,Yes
3,Sunny,Mild,High,False,Yes
4,Sunny,Cool,Normal,False,Yes
5,Sunny,Cool,Normal,True,No
6,Overcast,Cool,Normal,True,Yes
7,Rainy,Mild,High,False,No
8,Rainy,Cool,Normal,False,Yes
9,Sunny,Mild,Normal,False,Yes


In [5]:
df.keys()[:-1]

Index(['Outlook', 'Temp', 'Humidity', 'Windy'], dtype='object')

In [6]:
target = df.columns[-1]
target

'Play Golf'

In [7]:
for i in df.keys()[:-1]:
    unique_classes=df[i].unique()
    print(unique_classes)

['Rainy' 'Overcast' 'Sunny']
['Hot' 'Mild' 'Cool']
['High' 'Normal']
['False' 'True']


In [8]:
Class = df.keys()[-1]
unique_last=df[Class].unique()
unique_last

array(['No', 'Yes'], dtype=object)

In [9]:
"""

Oluşturmuş olduğumuz veri setinin entropisini hesaplayabilmek için öncelikle entropy last fonksiyonunu çağırıp parametre olarak 
df veri setini girmemiz gerekiyor. Daha sonra veri setinin label sütunu olan play golf feature'ının birbirinden farklı sınıflarını(yes no) 
bir liste halinde getirmeliyiz. Oluşturduğumuz for döngüsü sayesinde Unique_values daki no ve yes sırasıyla i parametresi şeklinde ele alınacak. 
Öncelikle Yes etiketinin geldiğini varsayalım. df[class].value_counts() bize play golf columns'undaki sınıfların frekans sayılarını verir. 
df[class].value_counts()[i] play golf daki toplam 'yes' etiketi sayısını(frekansını) verir. 
len(df[class]) metoduyla df veri setinin row sayısını ve aynı zamanda playgolftaki toplam veriyi bulmuş oluruz. 
İşte bu yaptığımız işlem yes için olasılık değeriydi bunun entropisi prob*np.log2(prob) şeklinde hesaplanmış oluyor.
Bu işlemler bittikten sonra for döngüsüyle 'No' yu i parametresi olarak alıyoruz.
Yes için yaptığımız işlemleri No için de yapıp entropy değişkenine atıyoruz ve toplamlarını alıyoruz. Böylelikle entropiyi hesaplamış oluyoruz.

𝐸𝑛𝑡𝑟𝑜𝑝𝑦(𝑥)=−∑(𝑃(𝑥=𝑘))∗Log2(𝑃(𝑥=𝑘)))

"""

def entropy_last(df):
    Class=df.keys()[-1]    
    unique_values=df[Class].unique()  
    entropy=0
    for i in unique_values:
        prob=df[Class].value_counts()[i]/len(df[Class]) 
        entropy+=-prob*np.log2(prob)
    return entropy

In [10]:
entropy_last(df)

0.9402859586706311

![](Entropy.jpg)

In [11]:
"""

İnformation gaini hesaplamadan önce aşağıdaki attiribute_entropy(df,attribute) fonksiyonunda 
feature('Oulook','Temp','Windy','Humidity')'ların entropisini hesaplamamız lazım.
Bunun için bir algoritma oluşturduk. best_for_split fonksiyonunda df veri setinin keyslerini alıp for döngüsünde sırasıyla attribute_entropy
fonksiyonuna gönderdiğimizde dönecek değerden bahsetmiştik.Peki biz bu fonksiyonda ne amaçladık ? Nasıl her feature'ın entropisini hesaplayabiliyoruz ?

attributes parametresine gelecek olan feature, örnek 'Outlook' olsun.
df[attribute].unique metoduyla Outlook feature'ındaki farklı sınıfları alırız. df.keys()[-1] metoduyla da df veri setimizin son sütunu 
aynı zamanda class labelı 'Play golf' columns değişkeninin ismini almış ve Class değişkenine atamış oluyoruz.

df[Class].unique() metoduyla 'Play golf' un farklı sınıf değerlerini unique_last değişkenine atıyoruz.


for döngüsünde unique_classes'daki birbirinden farklı sınıfları sırasıyla i parametresine atadık.tabi i değeri bir sonraki değere geçmeden önce
2.for döngüsüne de giriyor. Mesela 'Outlook' feature'ın farklı sınıflarının entropisini hesaplıyoruz. ['Rainy' 'Overcast' 'Sunny'] listesindeki sınıflar
feature'ın birbirinden farklı sınıflarıdır.

Rainy i parametresine gönderiliyor ve df verisetindeki Rainy sınıflı olanların class labeldaki ['No'] etiketinin sayısını tüm Rainy sayısına,
Rainy sınıflı olanların class labeldaki ['Yes'] etiketinin sayısını tüm Rainy sayısına bölünmesiyle entropi işlemi gerçekleşir.

Daha sonra aynı işlem diğer sınıflar olan ['Overcast','Sunny'] sınıfları için de geçerlidir.

En son feature'daki birbirinden farklı sınıfların entropilerini toplarız ve feature'ın genel entropisini bulmuş oluruz.

"""
def attribute_entropy(df,attribute):
    unique_classes=df[attribute].unique()
    Class=df.keys()[-1]
    unique_last=df[Class].unique()
    information=0
    for i in unique_classes:
        entropy=0
        for j in unique_last:
            num=len(df[attribute][df[attribute]==i][df[df.keys()[-1]]==j])
            den=len(df[attribute][df[attribute]==i])
            entropy+=-num/(den+eps)*np.log2(num/den+eps)
        information+=den/len(df)*entropy     
    return abs(information)

In [12]:
features_entropi = {}
for key in df.keys()[:-1]:
    features_entropi[key] = attribute_entropy(df,key)

In [13]:
features_entropi

{'Outlook': 0.6935361388961914,
 'Temp': 0.9110633930116756,
 'Humidity': 0.7884504573082889,
 'Windy': 0.892158928262361}

In [14]:
"""

Şimdi inforation gaini hesaplamak ve tree'nin root node kısmını bulmak için best_for_split adında bir fonksiyon oluşturduk ve df verisetimizi bu fonksiyona yolladık.
df veri setinin feature'larının information gainini hesaplamak için öncelikle keys() metoduyla featureların ismini alırız.
Play golf bir feature değildir çünkü o bir class labeldır.O yüzden df.keys()[:-1] işlemini uyguladık.
Burada for döngüsüyle key parametresi olarak sırasıyla gelecek olan her feature'ı attribute_entropy(df,key) 
fonksiyonuna göndereceğiz ve return değeri dönecek. entropy_last(df) fonksiyonunu üstteki hücrelerde kodlamıştık ve fonksiyonu
çağırdığımızda veri setimizin entropi değerini döndürür.
Daha sonra bir list metodu olan append() fonskiyonuyla sırasıyla gelecek her bir feature('outlook','Temp','Humidity','Windy')'ın
information gainini IG listesine eklemiş olacağız.

Tabi best_for_split fonksiyonunu kurmamızdaki temel amaç en yüksek information gaini olan feature'ı fonksiyon çağrıldığında döndürmesi.
Bunun için numpy kütüphanesinin np.argmax() kütüphanesini kullandık.

𝐸𝑛𝑡𝑟𝑜𝑝𝑦(𝑥)=−∑(𝑃(𝑥=𝑘))∗Log2(𝑃(𝑥=𝑘)))

𝐼𝑛𝑓𝑜𝑟𝑚𝑎𝑡𝑖𝑜𝑛𝐺𝑎𝑖𝑛(𝑓𝑒𝑎𝑡𝑢𝑟𝑒) = 𝐸𝑛𝑡𝑟𝑜𝑝𝑦(𝑑𝑎𝑡𝑎𝑠𝑒𝑡) − 𝐸𝑛𝑡𝑟𝑜𝑝𝑦(𝑓𝑒𝑎𝑡𝑢𝑟𝑒)

"""

def best_for_split(df):
    
    IG=[]
    for key in df.keys()[:-1]:
        IG.append(entropy_last(df)-attribute_entropy(df,key))
        
    return df.keys()[:-1][np.argmax(IG)]

![](IG.jpg)

In [15]:
IG=[]
for key in df.keys()[:-1]:
    IG.append(entropy_last(df)-attribute_entropy(df,key))
IG   

[0.24674981977443977,
 0.029222565658955535,
 0.15183550136234225,
 0.048127030408270155]

In [16]:
best_for_split(df)

'Outlook'

In [17]:
"""

reset_index() metoduyla, indexi varsayılan 0, 1, 2 vb. indexlerine sıfırlamamıza olanak sağlar.
ve tree oluştururken root node olan feature'ın classlarını tablo şeklinde göstermek istedik.

"""

def get_subtable(df,node,value):
    return df[df[node]==value].reset_index(drop=True)

In [18]:
a = get_subtable(df,'Outlook','Rainy')
a

,Outlook,Temp,Humidity,Windy,Play Golf
0,Rainy,Hot,High,False,No
1,Rainy,Hot,High,True,No
2,Rainy,Mild,High,False,No
3,Rainy,Cool,Normal,False,Yes
4,Rainy,Mild,Normal,True,Yes


In [19]:
get_subtable(df,'Outlook','Sunny')

,Outlook,Temp,Humidity,Windy,Play Golf
0,Sunny,Mild,High,False,Yes
1,Sunny,Cool,Normal,False,Yes
2,Sunny,Cool,Normal,True,No
3,Sunny,Mild,Normal,False,Yes
4,Sunny,Mild,High,True,No


In [20]:
get_subtable(df,'Outlook','Overcast')

,Outlook,Temp,Humidity,Windy,Play Golf
0,Overcast,Hot,High,False,Yes
1,Overcast,Cool,Normal,True,Yes
2,Overcast,Mild,High,True,Yes
3,Overcast,Hot,Normal,False,Yes


In [21]:
"""

Şimdi information gain değeri yüksek olan feature'ı bildiğimize göre ağacı python diliyle kabataslak
oluşturabiliriz. İnformation gaini en yüksek olan root node oluyordu.Daha sonra root node olan bu feature'ın
farklı class'larının alt tablolarını oluşturuyoruz.

Eğer alt tablolardaki class label'ların tüm verileri 'Yes' veya 'No' ise yani tek tip etiket varsa o tablonun etiketi bellidir.

Eğer tablodaki class label'ın verilerinin bazısı yes bazısı no veya class label etiketini belirleyecek feature yoksa tekrar bölme işlemi gerçekleşir
ve en iyi bölen feature belirlenir

Mesela Rainy sınıflarını barındıran alt tablonun tek etiketi olmadığından belirleyici feature bilmiyoruz bu yüzden 
en iyi bölen özelliğini bulmak için information gainini hesaplarız ve Humidity feature olduğu görülür.Humidity feature'ında
'High' sınıfında etiket 'No' , 'Normal' sınıfında ise etiket 'Yes' olduğu görülür. 

Overcast alt tablosundaki class label etiketlerinin hepsi 'Yes' olduğu için en iyi bölme metodunu kullanmaya gerek kalmadan etiket 'Yes'
olur.

Aynı mantık Sunny alt tablosunda da geçerlidir.

"""

def build_tree(df,tree=None):
    Class=df.keys()[-1]
    node=best_for_split(df)
    attr=df[node].unique()
    if tree is None:
        tree={}
        tree[node]={}
        for value in attr:
            subtable = get_subtable(df,node,value)
            clValue,counts = np.unique(subtable['Play Golf'],return_counts=True)                        
        
            if len(counts)==1:
                tree[node][value] = clValue[0]                                                    
            else:        
                tree[node][value] = build_tree(subtable)  
                   
    return tree

In [22]:
tree = build_tree(df)

In [23]:
tree

{'Outlook': {'Rainy': {'Humidity': {'High': 'No', 'Normal': 'Yes'}},
  'Overcast': 'Yes',
  'Sunny': {'Windy': {'False': 'Yes', 'True': 'No'}}}}

In [24]:
import pprint

In [25]:
pprint.pprint(tree)

{'Outlook': {'Overcast': 'Yes',
             'Rainy': {'Humidity': {'High': 'No', 'Normal': 'Yes'}},
             'Sunny': {'Windy': {'False': 'Yes', 'True': 'No'}}}}


![](Tree.jpg)

In [26]:
c,d = np.unique(df['Play Golf'],return_counts=True)

In [27]:
c

array(['No', 'Yes'], dtype=object)

In [28]:
d

array([5, 9], dtype=int64)

In [29]:
len(d)

2

In [30]:
data = {'Outlook':'Sunny','Temperature':'Cool','Humidity':'Normal','Windy':False}
inst = pd.Series(data)

In [31]:
inst

Outlook         Sunny
Temperature      Cool
Humidity       Normal
Windy           False
dtype: object

In [32]:
value = inst['Outlook']
value

'Sunny'

In [33]:
tree['Outlook'][value]

{'Windy': {'False': 'Yes', 'True': 'No'}}

In [34]:
"""

Sırada oluşturduğumuz bu decision tree algoritmasını test etmek var.
Örneğin {'Outlook':'Rainy','Temperature':'Cool','Humidity':'High','Windy':True}

alıştırmasının etiketini tahmin edin denildiğinde cevap ne olur ?

Etiket 'No' olur.Burada Root node'un class'larından biri olan 'Rainy' seçilmiş.
'Rainy' sınıflı alt tablonun belirleyici özelliği 'Humidity' değişkeniydi.

Humidity key'i 'Normal' ise etiket 'Yes' olur.
Humidity key'i 'High' ise etiket 'No' olur.

"""

def predict(inst,tree):
   
    for nodes in tree.keys():        
        
        value = inst[nodes]
        tree = tree[nodes][value]
        prediction = 0
            
        if type(tree) is dict:
            prediction = predict(inst, tree)
        else:
            prediction = tree
            break;                            
        
    return prediction

In [35]:
data = {'Outlook':'Rainy','Temperature':'Cool','Humidity':'Normal','Windy':True}
inst = pd.Series(data)

prediction = predict(inst,tree)

In [36]:
prediction

'Yes'

In [37]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier 
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn import tree

In [38]:
df = pd.read_csv('play_golf.csv')
df

,Outlook,Temp,Humidity,Windy,Play Golf
0,Rainy,Hot,High,False,No
1,Rainy,Hot,High,True,No
2,Overcast,Hot,High,False,Yes
3,Sunny,Mild,High,False,Yes
4,Sunny,Cool,Normal,False,Yes
5,Sunny,Cool,Normal,True,No
6,Overcast,Cool,Normal,True,Yes
7,Rainy,Mild,High,False,No
8,Rainy,Cool,Normal,False,Yes
9,Sunny,Mild,Normal,False,Yes


In [39]:
# Load the dataset
df = pd.read_csv('play_golf.csv') 

# Create label encoder instance
lb = LabelEncoder() 

# Create Numerical labels for classes
df['Play Golf_'] = lb.fit_transform(df['Play Golf'] ) 
df['Outlook_'] = lb.fit_transform(df['Outlook']) 
df['Temp_'] = lb.fit_transform(df['Temp'] ) 
df['Humidity_'] = lb.fit_transform(df['Humidity'] ) 
df['Windy_'] = lb.fit_transform(df['Windy'] ) 

# Split features and target variable
X = df[['Outlook_', 'Temp_', 'Humidity_', 'Windy_']] 
Y = df['Play Golf_']

# Instantiate a one hot encoder
enc = OneHotEncoder()

# Fit the feature set X
enc.fit(X)

# Transform X to onehot array 
onehotX = enc.transform(X).toarray()

onehotX, onehotX.shape, X.shape

(array([[0., 1., 0., 0., 1., 0., 1., 0., 1., 0.],
        [0., 1., 0., 0., 1., 0., 1., 0., 0., 1.],
        [1., 0., 0., 0., 1., 0., 1., 0., 1., 0.],
        [0., 0., 1., 0., 0., 1., 1., 0., 1., 0.],
        [0., 0., 1., 1., 0., 0., 0., 1., 1., 0.],
        [0., 0., 1., 1., 0., 0., 0., 1., 0., 1.],
        [1., 0., 0., 1., 0., 0., 0., 1., 0., 1.],
        [0., 1., 0., 0., 0., 1., 1., 0., 1., 0.],
        [0., 1., 0., 1., 0., 0., 0., 1., 1., 0.],
        [0., 0., 1., 0., 0., 1., 0., 1., 1., 0.],
        [0., 1., 0., 0., 0., 1., 0., 1., 0., 1.],
        [1., 0., 0., 0., 0., 1., 1., 0., 0., 1.],
        [1., 0., 0., 0., 1., 0., 0., 1., 1., 0.],
        [0., 0., 1., 0., 0., 1., 1., 0., 0., 1.]]),
 (14, 10),
 (14, 4))

In [40]:
X_train, X_test , y_train,y_test = train_test_split(onehotX, Y, test_size = 0.3, random_state = 42) 

In [41]:
from sklearn.metrics import confusion_matrix,accuracy_score

In [42]:
clf= DecisionTreeClassifier(criterion='entropy')
clf.fit(X_train,y_train) 

DecisionTreeClassifier(criterion='entropy')

In [43]:
y_test_pred = clf.predict(X_test)
y_test_pred

array([1, 0, 0, 1, 1])

In [44]:
y_train_pred = clf.predict(X_train)
y_train_pred

array([1, 1, 0, 0, 1, 0, 1, 1, 1])

In [45]:
print(accuracy_score(y_train, y_train_pred))
confusion_matrix(y_train, y_train_pred)

1.0


array([[3, 0],
       [0, 6]], dtype=int64)

In [46]:
from sklearn import metrics

In [47]:
# confusion matrix
confusion_matrix(y_test, y_test_pred)
print(confusion_matrix(y_test, y_test_pred))

# accuracy
print("Accuracy:", metrics.accuracy_score(y_test,y_test_pred))

[[1 1]
 [1 2]]
Accuracy: 0.6


In [48]:
from sklearn.tree import export_graphviz
from six import StringIO  
from IPython.display import Image  
import pydotplus, graphviz

In [49]:
dot_data = StringIO()

In [50]:
export_graphviz(clf, out_file=dot_data, filled=True, rounded=True,
                feature_names=df.columns, 
                class_names=['Yes', "No"])